# Приложение для чтения книг 

## Описание проекта

Провести анализ базы данных одного из сервисов для чтения книг и сделать предложения по созданию нового конкурентного продукта.

### Задание

- Посчитать количество книг, вышедших после 1 января 2000 года;
- Посчитать количество обзоров и среднюю оценку каждой книги;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры);
- Определить автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Предобработка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# Функция для вывода данных
def show(query):
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
books = '''
SELECT *
FROM books
'''
show(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
show(books).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [5]:
print('Дубликатов:', show(books).duplicated().sum())
print()
print('Пропусков:')
print(show(books).isna().sum())

Дубликатов: 0

Пропусков:
book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64


Таблица "books" содержит список из 1000 строк (книг). Пропусков нет, дубликатов нет, формат столбца "publication_date" не временной

In [6]:
authors = '''
SELECT *
FROM authors

'''
show(authors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
show(authors).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [8]:
print('Дубликатов:', show(authors).duplicated().sum())
print()
print('Пропусков:') 
show(authors).isna().sum()    

Дубликатов: 0

Пропусков:


author_id    0
author       0
dtype: int64

Таблица "authors" содержит список из 636 строк (авторов). Пропусков нет, дубликатов нет

In [9]:
publishers = '''
SELECT *
FROM publishers
'''
show(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
show(publishers).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [11]:
print('Дубликатов:', show(publishers).duplicated().sum())
print()
print('Пропусков:') 
show(publishers).isna().sum()

Дубликатов: 0

Пропусков:


publisher_id    0
publisher       0
dtype: int64

Таблица "publishers" содержит список из 340 строк (книгоиздателей). Пропусков нет, дубликатов нет

In [12]:
ratings = '''
SELECT *
FROM ratings
'''
show(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [13]:
show(ratings).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [14]:
print('Дубликатов:', show(ratings).duplicated().sum())
print()
print('Пропусков:')  
show(ratings).isna().sum()

Дубликатов: 0

Пропусков:


rating_id    0
book_id      0
username     0
rating       0
dtype: int64

Таблица "ratings" содержит список из 6456 строк (рейтингов пользователей). Пропусков нет, дубликатов нет

In [15]:
reviews = '''
SELECT *
FROM reviews
'''
show(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [16]:
show(reviews).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [17]:
print('Дубликатов:', show(reviews).duplicated().sum())
print()
print('Пропусков:')  
show(reviews).isna().sum()

Дубликатов: 0

Пропусков:


review_id    0
book_id      0
username     0
text         0
dtype: int64

Таблица "reviews" содержит список из 2793 строк (обзоров пользователей). Пропусков нет, дубликатов нет

## Исследовательский анализ данных

### Количество книг, выпущенных после 1 января 2000 года:

In [18]:
books_count = '''
SELECT COUNT(book_id) AS books_count 
FROM books 
WHERE publication_date >'2000-01-01'
'''
show(books_count)

,books_count
0,819


После 1 января 2000 года вышло 819 книг

### Количество пользовательских обзоров и средняя оценка для каждой книги:

In [19]:
# rating_reviews = '''
#SELECT books.title AS title,
 #      books.book_id AS id,
 #      ROUND(AVG(ratings.rating), 2) AS avg_rating,
 #      COUNT(DISTINCT reviews.text) AS reviews_count
#FROM books
#INNER JOIN ratings ON ratings.book_id = books.book_id
#INNER JOIN reviews ON reviews.book_id = books.book_id
#GROUP BY books.book_id
#ORDER by reviews_count DESC
#'''
#show(rating_reviews)

In [20]:
rating_reviews = '''
SELECT books.title AS title,
       books.book_id AS id,
       ROUND(AVG(ratings.rating), 2) AS avg_rating,
       COUNT(DISTINCT reviews.text) AS reviews_count
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.book_id
ORDER by reviews_count DESC
'''
show(rating_reviews)

,title,id,avg_rating,reviews_count
0,Twilight (Twilight #1),948,3.66,7
1,Water for Elephants,963,3.98,6
2,The Glass Castle,734,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.41,6
4,The Curious Incident of the Dog in the Night-Time,695,4.08,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.67,0
996,The Natural Way to Draw,808,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.00,0
998,Essential Tales and Poems,221,4.00,0



Больше всего пользовательских обзоров (7) было написано для книги "Twilight (Twilight #1)" с рейтингом 3,66.

### Издательство, которое издало наибольшее число книг толще 50 страниц (т.е. из анализа исключаются различные брошюры)

In [21]:
book_publisher = '''
SELECT publishers.publisher AS publisher_name,
       COUNT(books.book_id) AS books_published
FROM publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher_name
ORDER BY books_published DESC
'''
show(book_publisher).head()

,publisher_name,books_published
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Penguin Books - издательство, которое выпустило наибольшее число книг (42 книги) толще 50 страниц

### Автор с самой высокой средней оценкой среди книг с 50 и более пользовательскими оценками

In [22]:
top_author = '''
SELECT author,
       AVG(sub.avg_rating) as total_avg_rating
FROM 
    (SELECT author,
            books.book_id,
            AVG(rating) as avg_rating
     FROM authors
     LEFT JOIN books ON books.author_id = authors.author_id
     LEFT JOIN ratings ON ratings.book_id = books.book_id
     GROUP BY author,
              books.book_id
     HAVING COUNT(rating) >= 50) as sub
GROUP BY author
ORDER BY total_avg_rating DESC
'''
show(top_author)

,author,total_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


J.K. Rowling и Mary GrandPré (соавторство) - автор с самой высокой средней оценкой книг с 50 и более оценками.

14 из 636 представленных авторов имеют 50 и более оценок пользователей

 ### Среднее количество обзоров пользователей, которые поставили более чем по 50 оценок

In [23]:
avg_text = '''
SELECT ROUND(avg(count)) 
FROM 
    (SELECT COUNT(text)
     FROM reviews
     FULL OUTER JOIN 
                     (SELECT username,
                             COUNT(rating_id) as cnt
                      FROM ratings
                      GROUP BY username) AS ratings_count
     ON ratings_count.username = reviews.username
     WHERE cnt > 50
     GROUP BY reviews.username) as count;                                  
'''
show(avg_text)

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

## Общий вывод:
После 1 января 2000 года вышло 819 книг

Больше всего пользовательских обзоров (7) было написано для книги Twilight (Twilight #1)

Penguin Books - издательство, которое выпустило наибольшее число книг (42 книги) толще 50 страниц

J.K. Rowling/Mary GrandPré - авторы с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

Рекомендации:

Полученные результаты анализа базы данных, добытой нелегким трудом у конкурентов, с рейтингами и отзывами пользователей могут служить основой для формирования ассортимента нового приложения. 
В качестве книжного ассортимента для нового приложения рекомендуется выбрать книги авторов, указанных в сводной таблице 'top_author'. 
В первую очередь следует обратить внимание на книги, имеющие высокую популярность у читателей и проверенные временем, например такие, как "Гарри Поттер" или "Сумерки". 